In [3]:
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None
pd.set_option("display.precision", 3)
pd.set_option('future.no_silent_downcasting', True)

In [5]:
from football import striker
from football import main

affordable_fwds = striker.affordable_fwds_for_year(main.start_year)

for x in range(main.start_year+1, main.current_year+1):
  second_df = striker.affordable_fwds_for_year(x)
  affordable_fwds = pd.merge(
      left=affordable_fwds,
      right=second_df,
      how='outer',
      left_on=['first_name', 'second_name'],
      right_on=['first_name', 'second_name'],
  )

# Drop NA in any of two columns
# filters out unavailable now
# filter out newcomers with outlying results
last_year = main.current_year - 1
affordable_fwds.dropna(subset=[last_year, main.current_year], inplace=True)

affordable_fwds = affordable_fwds.apply(main.chance_of_playing_this_round, axis=1)
affordable_fwds.dropna(subset=["chance_to_play"], inplace=True)

unavailable_fwds = affordable_fwds[affordable_fwds["chance_to_play"] < 75]
affordable_fwds.drop(unavailable_fwds.index, inplace=True)

# # Show top 10 fwds w/ precision set to 1
affordable_fwds.sort_values(by=last_year, ascending=False, inplace=True)
affordable_fwds

,first_name,second_name,2020,2021,2022,2023,2024,chance_to_play
32,Danny,Ings,181.000,270.143,197.375,389.000,28.000,100
33,Danny,Welbeck,256.833,244.833,307.333,337.400,168.667,100
41,Dominic,Calvert-Lewin,179.438,256.200,583.000,309.429,251.500,100
90,Matheus,Santos Carneiro Da Cunha,NaN,NaN,480.500,203.167,230.000,100
95,Nicolas,Jackson,NaN,NaN,NaN,200.071,110.250,100
105,RaÃºl,JimÃ©nez,206.000,438.333,NaN,198.571,86.667,100
34,Darwin,NÃºÃ±ez Ribeiro,NaN,NaN,187.111,184.818,133.000,100
100,Ollie,Watkins,237.714,268.182,208.600,169.579,110.250,100
67,Jean-Philippe,Mateta,215.000,227.600,374.000,142.125,214.000,100
26,Chris,Wood,228.417,538.800,269.000,128.643,158.667,100


In [6]:
# Predict stat using previous data
from football import sklearn
predicted_fwds = affordable_fwds.copy().apply(sklearn.predict_this_year_linear, axis=1)

# Drop if prediction score is lower than 0.5 
# predicted_fwds.drop(predicted_fwds[predicted_fwds["prediction_linear_score"] < 0].index, inplace=True)
# Sort by prediction
predicted_fwds.sort_values(by="prediction_linear", ascending=False, inplace=True)
predicted_fwds.head()

,first_name,second_name,2020,2021,2022,2023,2024,chance_to_play,prediction_linear,prediction_linear_score
41,Dominic,Calvert-Lewin,179.438,256.200,583.000,309.429,251.500,100,511.210,-0.374
32,Danny,Ings,181.000,270.143,197.375,389.000,28.000,100,397.188,-1.123
33,Danny,Welbeck,256.833,244.833,307.333,337.400,168.667,100,362.650,-1.305
105,RaÃºl,JimÃ©nez,206.000,438.333,NaN,198.571,86.667,100,229.639,0.147
67,Jean-Philippe,Mateta,215.000,227.600,374.000,142.125,214.000,100,221.625,0.025


In [7]:
predicted_fwds_svr = predicted_fwds.apply(sklearn.predict_this_year_svr, axis=1)

# Drop if prediction score is lower than 0
predicted_fwds_svr.drop(predicted_fwds_svr[predicted_fwds_svr["prediction_svr_score"] < -0.5].index, inplace=True)
# predicted_fwds_svr.drop(predicted_fwds_svr[predicted_fwds_svr["prediction_svr"] == 0].index, inplace=True)
# Sort by prediction
predicted_fwds_svr.sort_values(by="prediction_svr", ascending=False, inplace=True)
predicted_fwds_svr.head()

,first_name,second_name,2020,2021,2022,2023,2024,chance_to_play,prediction_linear,prediction_linear_score,prediction_svr,prediction_svr_score
90,Matheus,Santos Carneiro Da Cunha,NaN,NaN,480.500,203.167,230.000,100,-74.167,-0.977,341.815,-0.077
41,Dominic,Calvert-Lewin,179.438,256.200,583.000,309.429,251.500,100,511.210,-0.374,283.079,-0.047
33,Danny,Welbeck,256.833,244.833,307.333,337.400,168.667,100,362.650,-1.305,282.797,-0.097
26,Chris,Wood,228.417,538.800,269.000,128.643,158.667,100,148.935,0.284,248.076,-0.005
32,Danny,Ings,181.000,270.143,197.375,389.000,28.000,100,397.188,-1.123,234.168,-0.029


# Good assisting striker

In [10]:
# Get current season stats to calculate performance
fwd_data = predicted_fwds_svr.copy()
fwd_data = fwd_data.apply(striker.expected_vs_actual_assists, axis=1)

# Drop if performes much worse
fwd_data.drop(fwd_data[fwd_data["overperformed_assists"] < -1].index, inplace=True)
fwd_data.dropna(how='all', inplace=True)
fwd_data

AttributeError: module 'football.striker' has no attribute 'expected_vs_actual_assists'

In [6]:
# mix in player stats
fwds = main.player_type_for_year(main.current_year, player_type="MID")
player_info = pd.merge(
      left=def_data,
      right=fwds,
      how='left',
      left_on=['first_name', 'second_name'],
      right_on=['first_name', 'second_name'],
  )
player_custom_info = player_info[['first_name', 'second_name', main.current_year, 'prediction_linear', 'prediction_svr', 'overperformed_assists', 'now_cost', 'goals_conceded', 'minutes', 'assists', 'total_points', 'influence', 'creativity', 'bonus', 'bps','selected_by_percent', ]]
player_custom_info.head()

,first_name,second_name,2024,prediction_linear,prediction_svr,overperformed_assists,now_cost,goals_conceded,minutes,assists,total_points,influence,creativity,bonus,bps,selected_by_percent
0,Emile,Smith Rowe,0.002,0.010,0.007,0.00,58,3,444,1,33,121.6,99.5,5,120,29.8
1,Cole,Palmer,0.008,0.002,0.006,-0.19,107,7,505,4,61,330.4,228.2,7,247,44.3
2,Pedro,Lomba Neto,0.004,0.007,0.005,-0.16,63,2,239,1,8,46.0,71.6,0,31,0.6
3,Adama,TraorÃ©,0.004,0.010,0.005,NaN,51,4,466,2,26,117.0,141.0,3,111,3.9
4,Harry,Wilson,0.000,0.004,0.004,-0.01,53,1,36,0,2,0.2,5.2,0,7,0.1
